In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
file_path = "Top 100 Private Colleges.2003.csv"

In [3]:
data = pd.read_csv(file_path)
data.head()

,Overall Rank,School,State,Undergrad. Enrollment,Admission Rate,*SAT or ACT,Student/faculty Ratio,4-year Grad. Rate,6-year Grad. Rate,Quality Rank,Total Costs,Cost After Need-based Aid,Need Met,Aid From Grants,Cost After Non-Need-Based Aid,Non-Need-Based Aid+,Average Debt,Cost Rank
0,1.0,California Institute of Technology,CA,939.0,21%,99/100%,3.0,71%,85%,10.0,"$32,682","$10,981",100%,93%,"$18,553",15%,"$10,244",4.0
1,2.0,Rice University,TX,2787.0,24%,89/92%,5.0,68%,89%,19.0,"$28,350","$14,779",100%,88%,"$22,418",34%,"$12,705",5.0
2,3.0,Williams College,MA,1985.0,23%,93/93%,8.0,89%,94%,1.0,"$36,550","$14,737",100%,89%,"$33,251",NaN,"$12,316",26.0
3,4.0,Swarthmore College,PA,1479.0,24%,94/98%,8.0,86%,92%,4.0,"$38,676","$17,386",100%,85%,"$11,404",2%,"$12,759",21.0
4,5.0,Amherst College,MA,1618.0,18%,94/92%,9.0,84%,94%,5.0,"$38,492","$14,453",100%,92%,"$33,411",NaN,"$11,544",38.0


## Data Cleaning

In [4]:
def get_nan_indices(df, col_name):
    all_idxs = set(range(len(df.index)))
    filled_idxs = set(df[col_name].dropna().index.tolist())
    return all_idxs - filled_idxs

nan_idxs = set()
for column in data.columns:
    nan_idxs = nan_idxs.union(get_nan_indices(data, column))
print(nan_idxs)

{2, 4, 6, 8, 9, 10, 11, 12, 14, 17, 18, 19, 81, 22, 87, 26, 29, 32, 33, 98, 100, 101, 36, 39, 40, 41, 42, 43, 53}


In [5]:
clean_data = data.copy()
clean_data.drop(clean_data.index[list(nan_idxs)], inplace=True)

### Further Cleaning

In [6]:
for column in clean_data.columns:
    print(column, set(clean_data[column]), sep = "\n", end="\n\n")

Overall Rank
{1.0, 2.0, 4.0, 6.0, 8.0, 14.0, 16.0, 17.0, 21.0, 22.0, 24.0, 25.0, 26.0, 28.0, 29.0, 31.0, 32.0, 35.0, 36.0, 38.0, 39.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 82.0, 84.0, 85.0, 86.0, 87.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 100.0}

School
{'Vanderbilt University', 'Taylor University', 'Whitman College', 'Rhodes College', 'Carnegie Mellon University', 'Kenyon College', 'Bowdoin College', 'Davidson College', 'Trinity College', 'Wheaton College', 'Lawrence University', 'New York University', 'Furman University', 'California Institute of Technology', 'The Colorado College', 'Union College', 'Skidmore College', 'Saint Louis University', 'Duke University', 'Rice University', 'Barnard College', 'DePauw University', 'Trinity University', "St. John's College", 'St. Olaf College', '

In [7]:
idxs = clean_data.index
na_vals = (clean_data['Need Met'] == "NA%").tolist()
na_idxs = []
for idx, val in zip(idxs, na_vals):
    if val:
        na_idxs.append(idx)
print(na_idxs)

[84]


In [8]:
clean_data.drop(na_idxs, inplace=True)


### Splitting SAT and ACT

In [9]:
col = clean_data['*SAT or ACT']
indx = clean_data.index
idxs = [] # indices of rows having incomplete values
for idx, val in zip(indx, col):
    if "/" not in val:
        idxs.append(idx)
    
print("Indexes with incomplete data ", idxs)
clean_data.drop(idxs, inplace=True)


Indexes with incomplete data  [38, 45, 46, 59, 60, 63, 64, 68, 69, 73, 78, 80, 85]


In [10]:
#function for converting sat or act to sat data in new column
def satconverter(x):
    if '/' in x:
        return float(x.split('/')[0])
#function for converting sat or act to act data in new column
def actconverter(x):
    if '/' in x:
        return float(x.split('/')[1].replace("%", ""))

In [11]:
clean_data['SAT'] = clean_data['*SAT or ACT'].apply(satconverter)
clean_data['ACT'] = clean_data['*SAT or ACT'].apply(actconverter)
clean_data.drop('*SAT or ACT', axis=1, inplace=True)

In [12]:
# Function for removing $ nad , signs.
def money_str_to_int(x):
    temp = x
    temp = temp.replace(',', '')
    temp = temp.replace('$', '')
    temp = int(temp)
    return temp

In [13]:
# Function for removing $ nad , signs.
def percent_str_to_int(x):
    temp = x
    temp = temp.replace('%', '')
    temp = int(temp)
    return temp

In [14]:
money_cols = ['Total Costs', 'Cost After Need-based Aid', 'Cost After Non-Need-Based Aid', 'Average Debt']

for col in money_cols:
    clean_data[col] = clean_data[col].apply(money_str_to_int)

In [15]:
percent_cols = ['Admission Rate', '4-year Grad. Rate', '6-year Grad. Rate',
                'Need Met', 'Non-Need-Based Aid+', 'Aid From Grants']

for col in percent_cols:
    clean_data[col] = clean_data[col].apply(percent_str_to_int)

In [16]:
clean_data.head()

,Overall Rank,School,State,Undergrad. Enrollment,Admission Rate,Student/faculty Ratio,4-year Grad. Rate,6-year Grad. Rate,Quality Rank,Total Costs,Cost After Need-based Aid,Need Met,Aid From Grants,Cost After Non-Need-Based Aid,Non-Need-Based Aid+,Average Debt,Cost Rank,SAT,ACT
0,1.0,California Institute of Technology,CA,939.0,21,3.0,71,85,10.0,32682,10981,100,93,18553,15,10244,4.0,99.0,100.0
1,2.0,Rice University,TX,2787.0,24,5.0,68,89,19.0,28350,14779,100,88,22418,34,12705,5.0,89.0,92.0
3,4.0,Swarthmore College,PA,1479.0,24,8.0,86,92,4.0,38676,17386,100,85,11404,2,12759,21.0,94.0,98.0
5,6.0,Webb Institute,NY,67.0,42,7.0,79,83,39.0,8079,5579,20,100,8079,100,5700,1.0,100.0,100.0
7,8.0,Washington and Lee University,VA,1750.0,31,11.0,86,89,30.0,30225,15452,99,87,22063,31,15634,9.0,89.0,89.0


# Q1. Normalize each attribute of college dataset into range 0 to 1 (Refer to slides to do min-max normalization). 

In [17]:
def normalise(df, col_name):
    min_val = min(df[col_name])
    max_val = max(df[col_name])
    
    range_ = max_val - min_val
    
    df[col_name] = (df[col_name] - min_val) / range_

In [18]:
normalised_data_frame = clean_data.copy()

norm_cols = ['Undergrad. Enrollment', 'Admission Rate', 
             'Student/faculty Ratio', '4-year Grad. Rate', '6-year Grad. Rate',
             'Total Costs','Cost After Need-based Aid', 'Need Met', 'Aid From Grants',
        'Cost After Non-Need-Based Aid', 'Non-Need-Based Aid+', 'Average Debt', 'SAT', 'ACT']

for col in norm_cols:
    normalise(normalised_data_frame, col)

In [19]:
normalised_data_frame.head()


,Overall Rank,School,State,Undergrad. Enrollment,Admission Rate,Student/faculty Ratio,4-year Grad. Rate,6-year Grad. Rate,Quality Rank,Total Costs,Cost After Need-based Aid,Need Met,Aid From Grants,Cost After Non-Need-Based Aid,Non-Need-Based Aid+,Average Debt,Cost Rank,SAT,ACT
0,1.0,California Institute of Technology,CA,0.044895,0.109375,0.000000,0.797753,0.629630,10.0,0.764995,0.233873,1.0000,0.904110,0.354043,0.141414,0.363044,4.0,0.980392,1.000000
1,2.0,Rice University,TX,0.140040,0.156250,0.142857,0.764045,0.777778,19.0,0.630298,0.398303,1.0000,0.835616,0.484688,0.333333,0.450260,5.0,0.784314,0.822222
3,4.0,Swarthmore College,PA,0.072697,0.156250,0.357143,0.966292,0.888889,4.0,0.951370,0.511170,1.0000,0.794521,0.112392,0.010101,0.452174,21.0,0.882353,0.955556
5,6.0,Webb Institute,NY,0.000000,0.437500,0.285714,0.887640,0.555556,39.0,0.000000,0.000000,0.0000,1.000000,0.000000,1.000000,0.202006,1.0,1.000000,1.000000
7,8.0,Washington and Lee University,VA,0.086650,0.265625,0.571429,0.966292,0.777778,30.0,0.688598,0.427440,0.9875,0.821918,0.472688,0.303030,0.554063,9.0,0.784314,0.755556


# Q2. Implement information gain analysis on the colleges dataset to find the gain of each attribute as covered in the class. 

In [20]:
# Three categories corresponding to low, medium, high
rank_categ = [(1, 33), (34, 67), (68, 100)]
norm_categ = [(0, 0.33), (0.34, 0.67), (0.68, 1)]

In [21]:
rank_cols = ['Overall Rank', 'Cost Rank', 'Quality Rank']

norm_cols = ['Undergrad. Enrollment', 'Admission Rate', 'Student/faculty Ratio', '4-year Grad. Rate', 
        '6-year Grad. Rate', 'Total Costs','Cost After Need-based Aid', 'Need Met', 'Aid From Grants',
        'Cost After Non-Need-Based Aid', 'Non-Need-Based Aid+', 'Average Debt', 'SAT', 'ACT']

In [22]:
def information(vals):
    info = 0
    s = sum(vals)
    for x in vals:
        p = x/s
        if p != 0:
            info -= p * math.log(p, 2)
    return info

In [23]:
def entropy(p, info):
    ent = 0
    for i in range(len(p)):
        ent += p[i] * info[i]
    return ent

In [24]:
states = normalised_data_frame['State'].unique()
print(states)

['CA' 'TX' 'PA' 'NY' 'VA' 'GA' 'NC' 'IL' 'ME' 'MD' 'MO' 'IN' 'CO' 'MN'
 'IA' 'OH' 'MA' 'SC' 'CT' 'TN' 'WA' 'RI' 'DC']


In [25]:
state_count = normalised_data_frame['State'].value_counts().tolist()

In [26]:
state_information = information(state_count)
print('Information = ', state_information)

Information =  4.045413340232901


In [27]:
total_records = len(normalised_data_frame)

In [28]:
for col in norm_cols:
    infos = [] # information
    probs = [] # probabilities
    
    data_col = normalised_data_frame[col]
    
    for low, hi in norm_categ:
        freq_list = [] # frequencies of records satisfying the category and state
        req_categ = data_col.between(low, hi, inclusive=True)
        for state in states:
            state_rec = req_categ.loc[normalised_data_frame['State'] == state].tolist()
            freq = sum(state_rec)
            freq_list.append(freq)
        infos.append(information(freq_list))
        probs.append(sum(freq_list) / total_records)
    
    # Calculate entropy for this column
    entrpy = entropy(probs, infos)
    gain = state_information - entrpy
    print("Gain({}) = {}".format(col, gain))

Gain(Undergrad. Enrollment) = 0.32092490940388974
Gain(Admission Rate) = 0.5517704556255469
Gain(Student/faculty Ratio) = 0.47370066625532337
Gain(4-year Grad. Rate) = 0.4288020592904811
Gain(6-year Grad. Rate) = 0.9006836830424207
Gain(Total Costs) = 0.41147245545127653
Gain(Cost After Need-based Aid) = 0.6279496275293859
Gain(Need Met) = 0.20969660534392753
Gain(Aid From Grants) = 0.6440307360837498
Gain(Cost After Non-Need-Based Aid) = 0.5432867778666317
Gain(Non-Need-Based Aid+) = 0.6031368307054676
Gain(Average Debt) = 0.543601539010218
Gain(SAT) = 0.8305841041020168
Gain(ACT) = 0.7980571038113808


In [29]:
for col in rank_cols:
    infos = [] # information
    probs = [] # probabilities
    
    data_col = normalised_data_frame[col]
    
    for low, hi in rank_categ:
        freq_list = [] # frequencies of records satisfying the category and state
        req_categ = data_col.between(low, hi, inclusive=True)
        for state in states:
            state_rec = req_categ.loc[normalised_data_frame['State'] == state].tolist()
            freq = sum(state_rec)
            freq_list.append(freq)
        infos.append(information(freq_list))
        probs.append(sum(freq_list) / total_records)
    
    # Calculate entropy for this column
    entrpy = entropy(probs, infos)
    gain = state_information - entrpy
    print("Gain({}) = {}".format(col, gain))

Gain(Overall Rank) = 0.6545869443348256
Gain(Cost Rank) = 0.6604397204024708
Gain(Quality Rank) = 0.6555037878503662
